In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interactive, Button, HBox, VBox, IntSlider, FloatSlider
import json

# ---unique--------------------------------

# Setting the range for Velocity
V_list = np.arange(0, 200, 10) / 3.6  # convert kph to m/s

# Function to create an interactive plot
def interactive_plot_func(l, A, N):  # V is replaced by A
    l = l / 1000  # convert mm to m
    A = A / 1e3  # convert x10^(-3) s^2/m^2 to s^2/m^2
    
    # Calculation of Gain G
    G_list = []
    for V in V_list:  # Loop over V values
        G = 1 / (1 + A * V**2) * V / (N * l)
        G_list.append(G)
    
    fig, ax = plt.subplots()
    ax.set_xlim(0, 200)
    ax.set_ylim(-25, -5)
    ax.set_xlabel("Velocity V [kph]")  # Changed x-axis label
    ax.set_ylabel("Yawgain G [dB]")  # Changed y-axis label
    ax.grid()

    # Preparing the DataFrame
    df_all = pd.DataFrame()
    df_all["V_values"] = V_list * 3.6  # convert m/s to kph

    # Plotting the graph
    ax.plot(V_list * 3.6, 20*np.log10(G_list), label=f"N={N}", color='red')  # Changed plot arguments and label

    # Saving to dataframe
    df_all[f"N={N}"] = G_list  # Changed dictionary key

    plt.show()
    return df_all


def save_data(button):
    df_all = interactive_plot_func(**{child.description.split(' ')[0]: child.value for child in interactive_plot.children[:-1]})
    df_all.to_csv('data.csv', index=False)
    
# ---partially unique-------------------------------- 

# define a dictionary that links the description to the name and unit
slider_parameters = {
    "l": {"name": "Wheelbase", "unit": "mm"},
    "A": {"name": "Stability Factor", "unit": "x10^(-3) s^2/m^2"},
    "N": {"name": "Gear Ratio"}
}

# Creating the sliders
spec_sliders = {
    "l": IntSlider(min=2000, max=3500, step=100, value=2700, description=f'l [{slider_parameters["l"]["unit"]}]', continuous_update=False),
    "A": FloatSlider(min=0.5, max=2, step=0.1, value=1.3, description=f'A [{slider_parameters["A"]["unit"]}]', continuous_update=False),
    "N": IntSlider(min=12, max=20, step=1, value=16, description=f'N', continuous_update=False)
}
condition_sliders = {
}

# ---common--------------------------------

interactive_plot = interactive(interactive_plot_func,
                               **spec_sliders, 
                               **condition_sliders)

# Function to define the operation of the Save button
def save_values(b):
    spec_slider_values = {
        key: {
            'name': slider_parameters[key]['name'],
            'value': slider.value,
            'unit': slider_parameters[key]['unit']
        }
        for key, slider in spec_sliders.items()
    }
    try:
        with open('spec_defaults.json', 'r+') as f:
            data = json.load(f)
            data.update(spec_slider_values)
            f.seek(0)
            f.write(json.dumps(data))
            f.truncate()
    except FileNotFoundError:
        with open('spec_defaults.json', 'w') as f:
            json.dump(spec_slider_values, f)
            
# Function to define the operation of the Load button
def load_values(b):
    try:
        with open('spec_defaults.json', 'r') as f:
            defaults = json.load(f)
            for key, slider in spec_sliders.items():
                if key in defaults:
                    slider.value = defaults[key]["value"]
    except FileNotFoundError:
        print("No default values found. Please save first.")

# create buttons
save_param_button = Button(description="Save parameters")
save_param_button.on_click(save_values)

load_param_button = Button(description="Load parameters")
load_param_button.on_click(load_values)

save_data_button = Button(description="Save data")
save_data_button.on_click(save_data)

# Displaying the UI elements
display(HBox([interactive_plot, VBox([save_param_button, load_param_button, save_data_button])]))